In [1]:
import datetime
import numpy as np
from matplotlib import pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
import talib as ta

In [127]:
win = pd.read_excel('INDICES.xlsx', sheet_name='WIN', date_parser='Data')

In [128]:
win = win.dropna()

In [129]:
win = win.sort_values('Data')

In [4]:
bop = ta.BOP(win.Open, win.Max, win.Min, win.Close)

In [131]:
tAlta = np.array([])
tBaixa = np.array([])
maxTend = 0
minTend = 0
tendencia = '0'
profit = pd.DataFrame(columns=['DataEnt','Entrada','DataSai','Saida', 'Operacao','Saldo'])
posicionado = 0 # 1 - Sim, 0 - Não
volMinOper = 500000000
vlrMinDifMedias = 100


In [130]:
win.columns

Index(['Data', 'Open', 'Max', 'Min', 'Close', 'Media9', 'Media21', 'Media200',
       'Vol'],
      dtype='object')

In [115]:
profit

,DataEnt,Entrada,DataSai,Saida,Operacao,Saldo


In [68]:
def getMinValue(vlr1 : float, vlr2 : float):
    if(vlr1 <= vlr2):
        return vlr1
    else:
        return vlr2
    
def getMaxValue(vlr1 : float, vlr2 : float):
    if(vlr1 >= vlr2):
        return vlr1
    else:
        return vlr2    

In [132]:
for i in win.index:
    tBaixa = np.append(tBaixa, win.Close[i])
    tAlta = np.append(tAlta, win.Close[i])
    if(win.Media21[i] > win.Media9[i]): #identificando a tendencia a partir da diferença das medias
        if(tendencia == 'Baixa'): # identificar alternancia de tendencias
            tBaixa = np.append(tBaixa, win.Close[i])
            if(posicionado == 0):
                if(win.Close[i] < getMinValue(minTend, maxTend) and (win.Media21[i] - win.Media9[i]) > vlrMinDifMedias and win.Vol[i] > volMinOper): #localizando oportunidade de compra a partir da comparacao do vlr atual e o fundo conhecido da tendencia
                    profit.loc[profit.index.size, ['DataEnt','Entrada','Operacao']] = win.Data[i], win.Close[i], 'Venda'
                    posicionado = 1
                    minTend = min(tBaixa)
                else:
                    minTend = min(tBaixa)
            elif(posicionado == 1):
                if(win.Close[i] < getMinValue(minTend, maxTend) and (win.Media21[i] - win.Media9[i]) > vlrMinDifMedias and win.Vol[i] > volMinOper):
                    minTend = min(tBaixa)
                else:
                    profit.loc[profit.index.size -1, ['DataSai','Saida']] = win.Data[i], win.Close[i]
                    posicionado = 0
                    minTend = min(tBaixa)
        else:
            tendencia = 'Baixa'
            maxTend = min(tAlta)
            minTend = win.Close[i]
            tBaixa = []
            i = i-1
    elif(win.Media21[i] < win.Media9[i]):
        if(tendencia == 'Alta'): # identificar alternancia de tendencias
            tAlta = np.append(tAlta, win.Close[i])
            if(posicionado == 0):
                if(win.Close[i] > getMaxValue(minTend, maxTend) and (win.Media9[i] - win.Media21[i]) > vlrMinDifMedias and win.Vol[i] > volMinOper): #localizando oportunidade de compra a partir da comparacao do vlr atual e o fundo conhecido da tendencia
                    profit.loc[profit.index.size, ['DataEnt','Entrada','Operacao']] = win.Data[i], win.Close[i], 'Compra'
                    posicionado = 1
                    maxTend = max(tAlta)
                else:
                    maxTend = max(tAlta)
            elif(posicionado == 1):
                if(win.Close[i] > getMaxValue(minTend, maxTend) and (win.Media9[i] - win.Media21[i]) > vlrMinDifMedias and win.Vol[i] > volMinOper):
                    maxTend = max(tAlta)
                else:
                    profit.loc[profit.index.size -1, ['DataSai','Saida']] = win.Data[i], win.Close[i]
                    posicionado = 0
                    maxTend = max(tAlta)
        else:
            tendencia = 'Alta'
            maxTend = win.Close[i]
            minTend = max(tBaixa)
            tAlta = []
            i = i-1
        

In [137]:
profit

,DataEnt,Entrada,DataSai,Saida,Operacao,Saldo
0,2020-12-16 13:18:00,115935,2020-12-16 13:19:00,115920,Venda,NaN
1,2020-12-16 13:20:00,115825,2020-12-16 13:21:00,115845,Venda,NaN
2,2020-12-16 14:58:00,116575,2020-12-16 15:04:00,116925,Compra,NaN
3,2020-12-16 15:12:00,116995,2020-12-16 15:17:00,117185,Compra,NaN
4,2020-12-16 15:18:00,117305,2020-12-16 15:19:00,117265,Compra,NaN
...,...,...,...,...,...,...
142,2021-01-11 17:14:00,122665,2021-01-11 17:16:00,122620,Venda,NaN
143,2021-01-11 17:17:00,122565,2021-01-11 17:18:00,122600,Venda,NaN
144,2021-01-11 17:47:00,123300,2021-01-11 17:48:00,123280,Compra,NaN
145,2021-01-11 17:50:00,123305,2021-01-11 17:51:00,123245,Compra,NaN
